# Auto-encoder

Author @ysbecca

First, pre-processing for segmentation:

- sample patches (eventually experiment with different resolutions/sizes)
- label patches as EM/C, SM, N (normal) in that order of priority based on annotated regions

- rotational/mirrored augmentation
- H&E realistic augmentation?

In [2]:
import tensorflow as tf
import numpy as np
import py_wsi
import cnn_helper as cn

## Set network parameters

In [ ]:
def model(x_image_, y_true_):
    # Encoder
    model, _ = cn.new_conv_layer(input=x_image_,
                                  num_input_channels=num_channels,
                                  filter_size=filter_size,
                                  num_filters=num_filters,
                                  use_pooling=True,
                                  max_pool_size=max_pool_size,
                                  use_relu=True)
    model, num_fc_features = cn.flatten_layer(model)
    model, _ = cn.new_fc_layer(input=model,          
                             num_inputs=num_fc_features,
                             num_outputs=fc_size,
                             use_relu=True)
    model = tf.nn.dropout(model, keep_prob)
    model, _ = cn.new_fc_layer(input=model,
                             num_inputs=fc_size,
                             num_outputs=num_classes,
                             use_relu=True)

    y_pred = tf.nn.softmax(model)
    y_pred_cls = tf.argmax(model, dimension=1)
    cost = tf.reduce_mean(tf.losses.softmax_cross_entropy(logits=model, onehot_labels=y_true_, weights=weights_))

    # Outputs to be returned to CPU
    return y_pred, y_pred_cls, cost
